## 💻ETL para Movies Dataset 🎥
En el siguiente notebook se presentan las transformaciones requeridas para obtener un dataset limpio y confiable el cual va ser cargado a un modelo de recomendación.


### Importación librerias 📜

Las librerias importadas son las necesarias para realizar con exito el proceso ETL, algunas librerias se eligen debido a las caracteristicas del dataset, por ejemplo el modulo **literal_eval** de la libreria **ast** se usa debido a que los datos de algunas columnas vienen como string pero en realidad representan una estructura de datos más compleja como un dict o un list.

In [1]:
# Importación de librerías requeridas.
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from ast import literal_eval # Importamos este modulo para convertir String a Dict y a List.

Ahora leemos los datos desde un archivo CSV usando el metodo read_csv que provee pandas. En este paso hay que tener cuidado con la ruta donde se encuentra almacenado el archivos csv, para este caso accedemos de forma local.

In [15]:
# Usando la función read_csv de Pandas leemos y almacenamos movies_dataset como un dataframe con el nombre movies_data.
movies_data = pd.read_csv('../datasets/movies_dataset.csv', low_memory=False)

Previsualizamos las 5 primeras filas del dataframe así cómo las 5 últimas.

In [3]:
# Con la ayuda del metodo head echamos un vistazo preeliminar al dataframe creado.
movies_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
# Con la ayuda del metodo tail echamos un vistazo preeliminar al dataframe creado.
movies_data.tail()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


Con el metodo info() echamos un vistazo al dataset completo.

In [9]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45466 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

En este punto he tomado la decisión de eliminar las columnas que no aportan información valiosa, en realidad esta decisión tiene sus bases en los requerimientos de más adelante especialmente en el procesamiento de los datos para las funciones y el modelo ML, pero deciso hacerlo de una vez para ir ahorrando procesamiento cuando se ejecuten las transfomaciones siguientes.

In [16]:
# Usando el metodo drop de Pandas borramos las columnas que se consideran inescesarias, se adiciona la opcion inplace=True para que los cambios se reflejen en el mismo dataframe.
movies_data.drop(columns=['video','imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)


### Valores Nulos🗑️
Una de las transformaciones solicitadas, adicional que hace parte fundamental de los ciclos ETL, es el manejo de datos nulos, para nuestro caso debemos cumplir los siguientes criterios:
* Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.
* Los valores nulos del campo release date deben eliminarse.

Estas transformaciones se llevan a cabo con el uso de funciones contenidas en pandas especificamente fillna() y dropna().


In [17]:
movies_data['revenue'] = movies_data['revenue'].fillna(0)
movies_data['budget'] = movies_data['budget'].fillna(0)
movies_data['release_date'].dropna(inplace=True)

### Desanidando 

Se procede a desanidar los campos de las columnas 'belongs_to_collection', 'genres', 'production_companies', 'production_countries' y 'spoken_languages'. El metodo usado es el resultado de varios pruebas realizadas con anterioridad para determinar cual es el mejor, inicialmente durante la exploración preeliminar se evidencia que los datos anidados vienen en tipo **str** y con el modulo **literal_eval** de la libreria **ast** se logra convertir a la forma ya sea **dict** o **list**, posteriormente se procede a desanidar y almacenar en dataframes independientes usando el modulo **json_normalize** de Pandas, este metodo se elige ya que se debe cumplir con rapidez con la tarea asignada y no vamos a reinventar la rueda, simplemente usamos lo que ya existe.

In [7]:
# Imprimimos el tipo un campo para cada columna que dice el diccionario de datos que están anidados para visualizar su estructura, luego imprimimos el typo de dato para evidenciar cómo viene formateado.
print(movies_data['belongs_to_collection'][2])
print(movies_data['genres'][2])
print(movies_data['production_companies'][2])
print(movies_data['production_countries'][2])

print(type(movies_data['belongs_to_collection'][2]))
print(type(movies_data['genres'][2]))
print(type(movies_data['production_companies'][2]))
print(type(movies_data['production_countries'][2]))

{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}
[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]
[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [8]:
'''Aplicamos una función Lambda a cada columna que viene anidada, dentro de la función lambda se aplica a cada campo la funcion
literal_eval para que trasnforme el string en su estrucutura adecuada ya sea dict o list, en el caso de encontrar un valor nulo se reemplaza con np.nanksk'''

movies_data['belongs_to_collection'] = movies_data['belongs_to_collection'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)
movies_data['genres'] = movies_data['genres'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)
movies_data['production_companies'] = movies_data['production_companies'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)
movies_data['production_countries'] = movies_data['production_countries'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)

### Creando la columna return 💰

En este punto se agrega una columna llamada **return** la cual se calcula con los campos **revenue** y **budget**, dividiendo estas dos últimas **revenue / budget**, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0. En una celda anterior cuando revisamos el tipo de dato de cada columna nos damos cuenta que la columna budget es de tipo object, por lo tanto debemos formatearla antes de proceder, adicional despues de algunas pruebas se evidencia que este campo trae información que no corresponde a un valor numerico represntado con string por lo tanto se usa una función lambda para remplazar esos valores por un '0', usando expresiones regulares ya que durante las pruebas se evidencia que el valor que trae es algo como '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'

In [18]:
movies_data['budget'] = movies_data['budget'].str.replace(r'/[a-zA-Z0-9]+\.jpg', '0', regex=True)
movies_data['budget'] = movies_data['budget'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)
movies_data['return'] = movies_data.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)

In [20]:
movies_data['return'].head()

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
Name: return, dtype: float64